# Gem-Flux MCP Server: Error Handling and Recovery

This notebook demonstrates common errors you may encounter and how to handle and recover from them.

## Error Categories

1. **ValidationError** - Invalid input data
2. **NotFoundError** - Model/media/compound/reaction not found
3. **InfeasibilityError** - Model cannot grow (gapfilling or FBA)
4. **DatabaseError** - Database loading or query issues
5. **LibraryError** - ModelSEEDpy or COBRApy failures

## Error Response Format

All errors follow a consistent JSON structure:
```json
{
  "success": false,
  "error_type": "ValidationError",
  "message": "Human-readable error description",
  "details": {...},
  "suggestion": "Actionable recovery suggestion"
}
```

## Setup

In [ ]:
# Import all tools
from gem_flux_mcp.tools.media_builder import build_media
from gem_flux_mcp.tools.build_model import build_model
from gem_flux_mcp.tools.gapfill_model import gapfill_model
from gem_flux_mcp.tools.run_fba import run_fba
from gem_flux_mcp.tools.compound_lookup import get_compound_name, search_compounds
from gem_flux_mcp.tools.reaction_lookup import get_reaction_name
from gem_flux_mcp.tools.list_models import list_models
from gem_flux_mcp.tools.delete_model import delete_model

# Import database and storage
from gem_flux_mcp.database.loader import load_compounds_database, load_reactions_database
from gem_flux_mcp.database.index import DatabaseIndex
from gem_flux_mcp.templates.loader import load_templates
from gem_flux_mcp.media.atp_loader import load_atp_media
from gem_flux_mcp.storage.models import clear_all_models
from gem_flux_mcp.storage.media import clear_all_media

# Import types
from gem_flux_mcp.types import (
    BuildMediaRequest,
    BuildModelRequest,
    GapfillModelRequest,
    RunFBARequest,
    GetCompoundNameRequest
)

# Import error types
from gem_flux_mcp.errors import (
    ValidationError,
    NotFoundError,
    InfeasibilityError,
    DatabaseError
)

from pathlib import Path
import json

print("✓ Imports successful")

## Initialize Database

In [ ]:
# Clear session
clear_all_models()
clear_all_media()

# Load database
database_dir = Path("../data/database")
compounds_df = load_compounds_database(str(database_dir / "compounds.tsv"))
reactions_df = load_reactions_database(str(database_dir / "reactions.tsv"))
db_index = DatabaseIndex(compounds_df, reactions_df)

# Load templates
templates = load_templates(str(Path("../data/templates")))
atp_media = load_atp_media()

print("✓ Database and templates loaded")

## Part 1: Validation Errors

### Invalid Compound IDs

Attempting to use nonexistent or malformed compound IDs.

In [ ]:
try:
    # Invalid compound IDs (don't exist in database)
    request = BuildMediaRequest(
        compounds=["cpd99999", "cpd88888", "cpd00027"],  # First two are invalid
        default_uptake=100.0
    )
    response = build_media(request, db_index)
except ValidationError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Invalid IDs: {e.details['invalid_ids']}")
    print(f"  Valid IDs: {e.details['valid_ids']}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error handled gracefully")

### Invalid Amino Acid Sequences

In [ ]:
try:
    # Protein sequence with invalid characters
    response = await build_model(
        protein_sequences={
            "prot1": "MKLVINLV",           # Valid
            "prot2": "MKXLINVAL*",         # Invalid: contains 'X' and '*'
            "prot3": "ABCDEFGHIJ"          # Invalid: contains 'B' and 'J'
        },
        template="GramNegative",
        annotate_with_rast=False
    )
except ValidationError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Invalid proteins: {e.details['invalid_proteins']}")
    print(f"  Valid alphabet: {e.details.get('valid_alphabet', 'ACDEFGHIKLMNPQRSTVWY')}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error handled gracefully")

### Empty Input

In [ ]:
try:
    # Empty compounds list
    request = BuildMediaRequest(
        compounds=[],
        default_uptake=100.0
    )
    response = build_media(request, db_index)
except ValidationError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error handled gracefully")

## Part 2: Not Found Errors

### Model Not Found

In [ ]:
try:
    # Try to run FBA on nonexistent model
    response = run_fba(
        model_id="nonexistent_model.draft",
        media_id="glucose_minimal_aerobic"
    )
except NotFoundError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Requested ID: {e.details['requested_id']}")
    print(f"  Available models: {e.details['available_models']}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error includes helpful list of available models")

### Compound Not Found

In [ ]:
try:
    # Try to lookup invalid compound
    request = GetCompoundNameRequest(compound_id="cpd99999")
    response = get_compound_name(request, db_index)
except NotFoundError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Compound ID: {e.details['compound_id']}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Suggests using search_compounds to find valid IDs")

## Part 3: Infeasibility Errors

### Infeasible FBA

Model cannot grow in the specified media (needs gapfilling).

In [ ]:
# First create a draft model
build_response = await build_model(
    protein_sequences={"prot1": "MKLVINLV"},
    template="Core",
    model_name="draft_model",
    annotate_with_rast=False
)
draft_model_id = build_response["model_id"]
print(f"✓ Created draft model: {draft_model_id}")

# Try to run FBA on ungapfilled model
try:
    fba_response = run_fba(
        model_id=draft_model_id,
        media_id="glucose_minimal_aerobic"
    )
except InfeasibilityError as e:
    print(f"\nError Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Model ID: {e.details['model_id']}")
    print(f"  Media ID: {e.details['media_id']}")
    print(f"  Status: {e.details['status']}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error explains model needs gapfilling")

### Gapfilling Infeasibility

Gapfilling cannot find reactions to enable growth.

In [ ]:
try:
    # Try to gapfill with very limited media (likely to fail)
    limited_media_request = BuildMediaRequest(
        compounds=["cpd00001"],  # Only water - no carbon source!
        default_uptake=100.0
    )
    limited_media_response = build_media(limited_media_request, db_index)
    limited_media_id = limited_media_response.media_id
    
    gapfill_response = gapfill_model(
        model_id=draft_model_id,
        media_id=limited_media_id,
        db_index=db_index,
        target_growth_rate=0.01,
        gapfill_mode="complete"
    )
except InfeasibilityError as e:
    print(f"Error Type: {e.error_type}")
    print(f"Message: {e.message}")
    print(f"\nDetails:")
    print(f"  Model ID: {e.details['model_id']}")
    print(f"  Media ID: {e.details['media_id']}")
    print(f"  Target growth: {e.details['target_growth']}")
    print(f"\nSuggestion: {e.suggestion}")
    print(f"\n✓ Error suggests using richer media or lower target")

## Part 4: Recovery Workflows

### Workflow 1: Search → Lookup → Build Media

In [ ]:
print("Scenario: User wants to create glucose media but doesn't know the ID\n")

# Step 1: Search for glucose
from gem_flux_mcp.types import SearchCompoundsRequest

search_request = SearchCompoundsRequest(query="glucose", limit=5)
search_response = search_compounds(search_request, db_index)

print(f"Step 1 - Search Results: {search_response.num_results} compounds")
for result in search_response.results[:3]:
    print(f"  {result.id}: {result.name}")

# Step 2: Get detailed info for D-Glucose
glucose_id = search_response.results[0].id
compound_request = GetCompoundNameRequest(compound_id=glucose_id)
compound_response = get_compound_name(compound_request, db_index)

print(f"\nStep 2 - Compound Details:")
print(f"  Name: {compound_response.name}")
print(f"  Formula: {compound_response.formula}")

# Step 3: Build media using found ID
media_request = BuildMediaRequest(
    compounds=[glucose_id, "cpd00007", "cpd00001"],
    default_uptake=100.0,
    custom_bounds={glucose_id: (-5.0, 100.0)}
)
media_response = build_media(media_request, db_index)

print(f"\nStep 3 - Media Created:")
print(f"  Media ID: {media_response.media_id}")
print(f"  Compounds: {media_response.num_compounds}")
print(f"\n✓ Workflow complete: Search → Lookup → Build Media")

### Workflow 2: Infeasible FBA → Gapfill → Retry FBA

In [ ]:
print("Scenario: Draft model can't grow → gapfill → FBA succeeds\n")

# We already have draft_model_id from earlier
print(f"Step 1 - Draft model: {draft_model_id}")
print(f"  Status: Cannot grow (infeasible FBA)")

# Step 2: Gapfill the model
print(f"\nStep 2 - Gapfilling model...")
gapfill_response = gapfill_model(
    model_id=draft_model_id,
    media_id="glucose_minimal_aerobic",
    db_index=db_index,
    target_growth_rate=0.01,
    gapfill_mode="complete"
)
gapfilled_model_id = gapfill_response["model_id"]

print(f"  Gapfilled model: {gapfilled_model_id}")
print(f"  Reactions added: {gapfill_response['num_reactions_added']}")
print(f"  Growth rate: {gapfill_response['growth_rate_after']:.3f} hr⁻¹")

# Step 3: Retry FBA on gapfilled model
print(f"\nStep 3 - Running FBA on gapfilled model...")
fba_response = run_fba(
    model_id=gapfilled_model_id,
    media_id="glucose_minimal_aerobic"
)

print(f"  Status: {fba_response['status']}")
print(f"  Growth rate: {fba_response['objective_value']:.3f} hr⁻¹")
print(f"  Active reactions: {fba_response['active_reactions']}")
print(f"\n✓ Recovery complete: Infeasible → Gapfill → Success")

### Workflow 3: Model Not Found → List Models → Use Correct ID

In [ ]:
print("Scenario: User forgets model ID → list models → find correct ID\n")

# Step 1: Error - wrong model ID
print("Step 1 - Attempted to use wrong model ID")
print("  Error: Model 'my_model.draft' not found")

# Step 2: List all models to find correct ID
print("\nStep 2 - Listing all models in session...")
from gem_flux_mcp.types import ListModelsRequest

list_request = ListModelsRequest()
list_response = list_models(list_request)

print(f"  Available models: {list_response.total_models}\n")
for model in list_response.models:
    print(f"  {model['model_id']}")
    print(f"    Name: {model['model_name']}")
    print(f"    State: {model['state']}")

# Step 3: Use correct ID
if list_response.models:
    correct_id = list_response.models[0]['model_id']
    print(f"\nStep 3 - Using correct model ID: {correct_id}")
    print(f"\n✓ Recovery complete: Found correct model ID")

## Part 5: Error Prevention Best Practices

### 1. Validate Inputs Before Tool Calls

In [ ]:
def validate_compound_ids(compound_ids, db_index):
    """Validate compound IDs before using them."""
    valid_ids = []
    invalid_ids = []
    
    for cpd_id in compound_ids:
        if db_index.compound_exists(cpd_id):
            valid_ids.append(cpd_id)
        else:
            invalid_ids.append(cpd_id)
    
    return valid_ids, invalid_ids

# Example usage
test_ids = ["cpd00027", "cpd99999", "cpd00001"]
valid, invalid = validate_compound_ids(test_ids, db_index)

print(f"Validation Results:")
print(f"  Valid: {valid}")
print(f"  Invalid: {invalid}")

if invalid:
    print(f"\n⚠️ Warning: Found invalid compound IDs")
    print(f"  Fix before calling build_media")
else:
    print(f"\n✓ All compound IDs valid")

### 2. Check Model Exists Before Operations

In [ ]:
def check_model_exists(model_id):
    """Check if model exists in session."""
    from gem_flux_mcp.storage.models import model_exists
    return model_exists(model_id)

# Example usage
test_model_id = "nonexistent.draft"

if check_model_exists(test_model_id):
    print(f"✓ Model exists: {test_model_id}")
else:
    print(f"⚠️ Model not found: {test_model_id}")
    print(f"  Listing available models...")
    
    list_request = ListModelsRequest()
    list_response = list_models(list_request)
    
    if list_response.models:
        print(f"  Available: {[m['model_id'] for m in list_response.models]}")
    else:
        print(f"  No models in session - create one first")

### 3. Use Predefined Media to Avoid Errors

In [ ]:
# Predefined media are guaranteed to be valid
predefined_media_names = [
    "glucose_minimal_aerobic",
    "glucose_minimal_anaerobic",
    "pyruvate_minimal_aerobic",
    "pyruvate_minimal_anaerobic"
]

print("Predefined Media (Always Available):")
for media_name in predefined_media_names:
    print(f"  ✓ {media_name}")

print(f"\nRecommendation: Use predefined media for reliability")
print(f"  No compound ID lookups needed")
print(f"  No validation errors")
print(f"  Based on standard protocols")

## Summary

This notebook demonstrated:

1. **Validation Errors**:
   - Invalid compound IDs
   - Invalid amino acid sequences
   - Empty inputs

2. **Not Found Errors**:
   - Model not found (with available models list)
   - Media not found
   - Compound not found (with search suggestions)

3. **Infeasibility Errors**:
   - Infeasible FBA (model needs gapfilling)
   - Gapfilling failure (media too limited)

4. **Recovery Workflows**:
   - Search → Lookup → Build Media
   - Infeasible FBA → Gapfill → Retry
   - Model Not Found → List → Use Correct ID

5. **Error Prevention**:
   - Validate inputs before tool calls
   - Check model existence
   - Use predefined media

## Key Takeaways

- **Consistent error format**: All errors include type, message, details, suggestion
- **Actionable suggestions**: Every error includes recovery guidance
- **Helpful details**: Errors include available IDs/models when relevant
- **Graceful handling**: Tools never crash, always return structured errors

## Best Practices

1. **Read error messages**: They contain specific guidance
2. **Check suggestions**: Follow recommended recovery steps
3. **Use search tools**: Find valid IDs before building media/models
4. **List models/media**: See what's available in session
5. **Use predefined media**: Avoid compound ID errors
6. **Gapfill before FBA**: Draft models typically need gapfilling

## Next Steps

- Implement error handling in your workflows
- Use try/except blocks for robust code
- Log errors for debugging
- Test error cases proactively

See other notebooks:
- `01_basic_workflow.ipynb` - Complete modeling workflow
- `02_database_lookups.ipynb` - Explore compound/reaction databases
- `03_session_management.ipynb` - Manage models and media